In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp -r "/content/drive/My Drive/datasets/Covid19-dataset" "/content/"

In [3]:
import tensorflow as tf
from keras import models,layers
import matplotlib.pyplot as plt

In [4]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 50

In [5]:
dataset_train = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/datasets/Covid19-dataset/train",
    shuffle = True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

Found 251 files belonging to 3 classes.


In [6]:
dataset_test = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/datasets/Covid19-dataset/test",
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

Found 66 files belonging to 3 classes.


In [7]:
class_names = dataset_train.class_names

In [8]:
class_names

['Covid', 'Normal', 'Viral Pneumonia']

In [9]:
len(dataset_train)

8

In [10]:
for image_batch, label_batch in dataset_train.take(1):
  print(image_batch.shape)
  print(label_batch)

(32, 256, 256, 3)
tf.Tensor([0 1 0 2 0 0 2 1 2 2 2 0 0 1 0 2 2 1 0 0 1 2 1 0 0 0 0 0 1 2 1 2], shape=(32,), dtype=int32)


In [11]:
dataset_train = dataset_train.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)

dataset_test = dataset_test.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)

In [13]:
resize_and_rescale = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE),
    tf.keras.layers.experimental.preprocessing.Rescaling(1.0/256)
])

In [14]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)
])

In [15]:
input_shape = (BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,CHANNELS)
n_classes = 5
model = models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32, (3,3), activation = 'relu',input_shape = input_shape),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, kernel_size = (3,3), activation = 'relu'),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, kernel_size = (3,3), activation = 'relu'),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, (3,3), activation = 'relu'),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, (3,3), activation = 'relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation = 'relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(64,activation = 'relu'),
    layers.Dense(n_classes, activation = 'softmax')
])

model.build(input_shape = input_shape)

In [16]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (32, 256, 256, 3)         0         
                                                                 
 sequential_2 (Sequential)   (32, 256, 256, 3)         0         
                                                                 
 conv2d (Conv2D)             (32, 254, 254, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (32, 127, 127, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (32, 125, 125, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (32, 62, 62, 64)         0         
 2D)                                                  

In [17]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
    metrics = ['accuracy']
)

In [ ]:
history = model.fit(
    dataset_train,
    batch_size = BATCH_SIZE,
    verbose=1,
    epochs = EPOCHS
)

Epoch 1/50
8/8 [==============================] - 47s 5s/step - loss: 1.3269 - accuracy: 0.3426
Epoch 2/50
8/8 [==============================] - 37s 5s/step - loss: 1.1299 - accuracy: 0.4064
Epoch 3/50
8/8 [==============================] - 39s 5s/step - loss: 1.1030 - accuracy: 0.3944
Epoch 4/50
8/8 [==============================] - 42s 5s/step - loss: 1.0842 - accuracy: 0.4064
Epoch 5/50
8/8 [==============================] - 41s 5s/step - loss: 1.0548 - accuracy: 0.5219
Epoch 6/50
8/8 [==============================] - 37s 5s/step - loss: 1.0292 - accuracy: 0.4462
Epoch 7/50
8/8 [==============================] - 39s 5s/step - loss: 0.9874 - accuracy: 0.4622
Epoch 8/50
8/8 [==============================] - 40s 5s/step - loss: 1.0443 - accuracy: 0.4064
Epoch 9/50
8/8 [==============================] - 37s 5s/step - loss: 0.9154 - accuracy: 0.5618
Epoch 10/50
8/8 [==============================] - 39s 5s/step - loss: 0.8258 - accuracy: 0.5976
Epoch 11/50
8/8 [======================

In [21]:
scores = model.evaluate(dataset_test)

3/3 [==============================] - 6s 618ms/step - loss: 0.2098 - accuracy: 0.9242


In [22]:
scores

[0.2097557783126831, 0.9242424368858337]

In [19]:
model.save("/content/drive/MyDrive/hackathon/x-ray")

In [20]:
model.save("/content/drive/MyDrive/hackathon/x-ray/xray_model.h5")

In [24]:
import os
model.save("/Users/aryanpatel/Downloads/hackathon/models")